# Dubai — Quantum accurate bond inference and partial charge calculations

See [the tutorial](/Tutorials/ligand_bond_inference_and_partial_charge_calculation.ipynb).

In [ ]:
# |hide
WORK_DIR = '~/qdx/dubai_quickstart/'
!rm -r $WORK_DIR || true
!mkdir -p $WORK_DIR
%cd -q $WORK_DIR

In [ ]:
import json
from pathlib import Path

import requests
import rush

# Get our client, for calling modules and using the rush API
# set the RUSH_TOKEN  environment variables
client = rush.build_blocking_provider_with_functions()

# Convert aspirin to a QDXF file so we can use it for this demo
SMILES_STRING = "CC(=O)OC1=CC=CC=C1C(=O)O"
SDF_LINK = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{SMILES_STRING}/record/SDF?record_type=3d"

file_path = client.workspace / "aspirin.sdf"
with open(file_path, "wb") as f:
    f.write(requests.get(SDF_LINK).content)

(ligand,) = client.convert(
    "SDF", Path(file_path)
)

ligand = json.load(open(ligand.download(), "r"))

# Remove connectivity
EXPECTED_CONNECTIVITY = ligand[0]["topology"]["connectivity"]
ligand = ligand[0]
del ligand["topology"]["connectivity"]

ligand["topology"]["fragment_multiplicities"] = [1]

# Arguments
DUBAI_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
LIGAND_FILEPATH = client.workspace / "aspirin.qdxf.json"
json.dump(ligand, open(LIGAND_FILEPATH, "w"))

# Run the inference
(ligand_with_bonds,) = client.dubai(
    LIGAND_FILEPATH, resources=DUBAI_RESOURCES
)

output_ligand = json.load(ligand_with_bonds.download().open())

for expected_bond, outputted_bond in zip(
    EXPECTED_CONNECTIVITY, output_ligand["topology"]["connectivity"]
):
    # Check start atoms are the same
    assert expected_bond[0] == outputted_bond[0]
    # Check ending atoms are the same
    assert expected_bond[1] == outputted_bond[1]
    # NB: we don't check the third item of the bond -- the bond type. This is
    # because Dubai accurately outputs ring bonds as a specific 'RINGBOND' type,
    # whereas SDF aspirin was interleaving single and double bonds.
print("Bond inference performed correctly!")

2024-03-16 10:16:20,028 - rush - INFO - Argument bd9a2e88-3520-4891-9935-d34a23558de4 is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:16:36,237 - rush - INFO - Argument bd9a2e88-3520-4891-9935-d34a23558de4 is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:16:53,447 - rush - INFO - Argument bd9a2e88-3520-4891-9935-d34a23558de4 is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:16:54,517 - rush - INFO - Argument bd9a2e88-3520-4891-9935-d34a23558de4 is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-03-16 10:17:10,128 - rush - INFO - Argument 242c61cf-40f2-44ad-9eda-dc62b9a8bbb6 is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:17:13,348 - rush - INFO - Argument 242c61cf-40f2-44ad-9eda-dc62b9a8bbb6 is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:23:22,896 - rush - INFO - Argument 242c61cf-40f2-44ad-9eda-dc62b9a8bbb6 is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:23:23,972 - rush - INFO - Argument 242c61cf-40f2-44ad-9eda-dc62b9a8bbb6 is now ModuleInstanceStatus.RUNNING
2024